In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score,accuracy_score,roc_auc_score,log_loss
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

from hyperopt import fmin, tpe, hp,space_eval,rand,Trials,partial,STATUS_OK

import random

In [ ]:
train_data = pd.read_csv('oof_cbt_12.csv')
test_data = pd.read_csv('test_cbt_12.csv')

train_data = pd.concat([train_data,pd.read_csv('oof_lgb_12.csv')],axis=1)
test_data =pd.concat([test_data,pd.read_csv('test_lgb_12.csv')],axis=1)

label = pd.read_csv('train_label.csv')
y = label.label.values

original_cols = train_data.columns

In [ ]:
space = {}
for idx,col in enumerate(original_cols):
    space['w'+str(10+idx)] = hp.uniform('w'+str(10+idx),-1,3)

In [ ]:
def wrapper(params):
    return -(roc_auc_score(y,np.dot(train_data[original_cols].values, np.array(list(params.values())))))

In [ ]:
#搜索oof加权后auc最优的权重组合

trials = Trials()
%time best = fmin(fn=wrapper, space=space, algo=tpe.suggest, max_evals=3000, trials=trials)
print(best)

In [ ]:
import datetime
now = datetime.datetime.now()
submit = pd.read_csv('submission.csv')
submit[['label']] = np.dot(test_data.values,np.array(list(best.values())))

mms = MinMaxScaler(feature_range=(0, 0.999999))
submit[['label']] = mms.fit_transform(submit[['label']])

submit.to_csv("submit/avg_{0}-{1}-{2}_{3}-{4}.csv".format(now.year,now.month,now.day,now.hour,now.minute),index=False)